## Time-Series modeling

#### Importing library

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import category_encoders as ce

#### Importing dataset

In [4]:
df = pd.read_csv("dataset.csv")